In [ ]:
# without yolo
# Import the required libraries
import numpy as np
import cv2
import tim
import datetime
import os
from collections import deque
from twilio.rest import Client

# Ensure the 'outputs' directory exists
if not os.path.exists("outputs"):
    os.makedirs("outputs")

# Define the kernel for morphological operations
kernel = np.ones((5,5), np.uint8)

def is_person_present(frame, thresh=1100):
    global foog
    
    # Apply background subtraction
    fgmask = foog.apply(frame)

    # Get rid of the shadows
    ret, fgmask = cv2.threshold(fgmask, 250, 255, cv2.THRESH_BINARY)

    # Apply some morphological operations to make sure you have a good mask
    fgmask = cv2.dilate(fgmask, kernel, iterations=4)

    # Detect contours in the frame
    contours, _ = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Check if there was a contour and the area is above a threshold
    if contours and cv2.contourArea(max(contours, key=cv2.contourArea)) > thresh:
        # Get the max contour
        cnt = max(contours, key=cv2.contourArea)

        # Draw a bounding box around the detected person
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        cv2.putText(frame, 'Person Detected', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1, cv2.LINE_AA)

        return True, frame
    else:
        return False, frame

def send_message(body, info_dict):
    """Sends an SMS alert using Twilio API"""
    account_sid = info_dict['account_sid']
    auth_token = info_dict['auth_token']

    client = Client(account_sid, auth_token)

    message = client.messages.create(
        to=info_dict['your_num'],
        from_=info_dict['trial_num'],
        body=body
    )

# Initialize OpenCV Window
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)

# Load Video File
cap = cv2.VideoCapture('sample_video.mp4')

In [ ]:
# video recording happning+ ip address also happning
import numpy as np
import cv2
import time
import datetime
import os
from collections import deque
from twilio.rest import Client

# Ensure 'outputs' directory exists
if not os.path.exists("outputs"):
    os.makedirs("outputs")

# Check required files
for file in ["yolov4.weights", "yolov4.cfg", "coco.names"]:
    if not os.path.exists(file):
        print(f"Error: {file} not found!")
        exit()

# Load YOLO model
weights_path = "yolov4.weights"
config_path = "yolov4.cfg"
labels_path = "coco.names"

with open(labels_path, "r") as f:
    labels = f.read().strip().split("\n")

net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
layer_names = net.getLayerNames()
out_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Detect persons using YOLO
def detect_person_yolo(frame, conf_threshold=0.5, nms_threshold=0.4):
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(out_layers)

    boxes, confidences = [], []
    person_detected = False
    
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if labels[class_id] == "person" and confidence > conf_threshold:
                box = detection[0:4] * np.array([width, height, width, height])
                (centerX, centerY, w, h) = box.astype("int")
                x = int(centerX - w / 2)
                y = int(centerY - h / 2)
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                person_detected = True
    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f'Person {confidences[i]:.2f}', (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    print(f"Person detected: {person_detected}")
    return person_detected, frame

# Twilio SMS function
def send_message(body, info_dict):
    account_sid = info_dict['account_sid']
    auth_token = info_dict['auth_token']
    client = Client(account_sid, auth_token)
    message = client.messages.create(
        to=info_dict['your_num'], from_=info_dict['trial_num'], body=body

In [2]:
# sample_video  code for video saving  
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                person_detected = True
    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f'Person {confidences[i]:.2f}', (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    print(f"Person detected: {person_detected}")
    return person_detected, frame

# Twilio SMS function
def send_message(body, info_dict):
    account_sid = info_dict['account_sid']
    auth_token = info_dict['auth_token']
    client = Client(account_sid, auth_token)
    message = client.messages.create(
        to=info_dict['your_num'], from_=info_dict['trial_num'], body=body
    )

# Load Sample Video
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
video_path = "sample_video.mp4"  # Change to actual path of the sample video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open sample video file!")
    exit()

width, height = int(cap.get(3)), int(cap.get(4))

# Detection Variables
status = False
patience = 7
detection_thresh = 15
de = deque([False] * detection_thresh, maxlen=detection_thresh)
initial_time, start_time = None, time.time()
fps, frame_counter = 0, 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("End of video reached.")
        break

    detected, annotated_image = detect_person_yolo(frame)
    de.appendleft(detected)
    print(f"Detection count in deque: {sum(de)}")

    if sum(de) == detection_thresh and not status:
        status = True
        entry_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")
        output_video_path = f'outputs/{entry_time}.mp4'
        out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'XVID'), 15.0, (width, height))
        print(f"Status set to True at {entry_time}")
        print(f"VideoWriter initialized at {output_video_path}")

    if status and not detected:
        if sum(de) > (detection_thresh / 2):
            if initial_time is None:
                initial_time = time.time()
        elif initial_time is not None and time.time() - initial_time >= patience:
            status = False
            exit_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")
            out.release()
            print(f"Recording stopped at {exit_time}")
            initial_time = None
    
    if status:
        print("Writing frame to video...")
        out.write(annotated_image)

    cv2.imshow('frame', annotated_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
if status:
    out.release()

IndentationError: unindent does not match any outer indentation level (<string>, line 6)

In [ ]:
import numpy as np
import cv2
import time
import datetime
import os
from collections import deque
from twilio.rest import Client

# Ensure 'outputs' directory exists
if not os.path.exists("outputs"):
    os.makedirs("outputs")

# Check required files
for file in ["yolov4.weights", "yolov4.cfg", "coco.names"]:
    if not os.path.exists(file):
        print(f"Error: {file} not found!")
        exit()

# Load YOLO model
weights_path = "yolov4.weights"
config_path = "yolov4.cfg"
labels_path = "coco.names"

with open(labels_path, "r") as f:
    labels = f.read().strip().split("\n")

net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
layer_names = net.getLayerNames()
out_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Detect persons using YOLO
def detect_person_yolo(frame, conf_threshold=0.5, nms_threshold=0.4):
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(out_layers)

    boxes, confidences = [], []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if labels[class_id] == "person" and confidence > conf_threshold:
                box = detection[0:4] * np.array([width, height, width, height])
                (centerX, centerY, w, h) = box.astype("int")
                x = int(centerX - w / 2)
                y = int(centerY - h / 2)
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    person_detected = False

    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f'Person {confidences[i]:.2f}', (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            person_detected = True

    return person_detected, frame

# Twilio SMS function
def send_message(body, info_dict):
    account_sid = info_dict['account_sid']
    auth_token = info_dict['auth_token']
    client = Client(account_sid, auth_token)
    message = client.messages.create(
        to=info_dict['your_num'], from_=info_dict['trial_num'], body=body
    )

# Load Video
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)

#  IP Camera URL
ip_camera_url ="http://192.168.130.118:8080/video"
cap = cv2.VideoCapture(ip_camera_url)

if not cap.isOpened():
    print("Error: Could not open IP camera! Retrying...")
    time.sleep(2)
    cap = cv2.VideoCapture(ip_camera_url)

if not cap.isOpened():
    print("Error: Camera connection failed! Check your network & IP camera settings.")
    exit()

cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
width, height = int(cap.get(3)), int(cap.get(4))

# Read credentials
if os.path.exists('credentialss.txt'):
    with open('credentialss.txt', 'r') as myfile:
        data = myfile.read()
    info_dict = eval(data)
else:
    print("Error: credentialss.txt not found!")
    exit()

# Detection Variables
status = False
patience = 7
detection_thresh = 15
de = deque([False] * detection_thresh, maxlen=detection_thresh)
initial_time, start_time = None, time.time()
fps, frame_counter = 0, 0
out = None  

while True:
    start_time = time.time()
    ret, frame = cap.read()
    if not ret:
        print("Error: Frame not received! Retrying...")
        cap.release()
        cap = cv2.VideoCapture(ip_camera_url)
        continue

    detected, annotated_image = detect_person_yolo(frame)
    de.appendleft(detected)

    if sum(de) == detection_thresh and not status:
        status = True
        entry_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")
        video_path = f'outputs/{entry_time}.mp4'
        frame_size = (frame.shape[1], frame.shape[0])
        out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), 15.0, frame_size)
        print(f"Recording started: {video_path}")

    if status:
        print("Writing frame to video...")
        out.write(annotated_image)

    if status and not detected:
        if sum(de) > (detection_thresh / 2):
            if initial_time is None:
                initial_time = time.time()
        elif initial_time is not None and time.time() - initial_time >= patience:
            status = False
            exit_time = datetime.datetime.now().strftime("%A, %I:%M:%S %p %d %B %Y")
            print(f"Recording stopped: {video_path}")
            out.release()
            initial_time = None
            body = f"Alert: A Person Entered at {entry_time} and Left at {exit_time}"
            send_message(body, info_dict)

    fps = frame_counter / (time.time() - start_time + 1e-5)
    frame_counter += 1
    cv2.putText(annotated_image, f'FPS: {fps:.2f}', (510, 450), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 40, 155), 2)
    cv2.imshow('frame', annotated_image)
    if cv2.waitKey(30) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
if status:
    out.release()


Recording started: outputs/Monday, 11:13:50 AM 07 April 2025.mp4
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...
Writing frame to video...

In [ ]:
from collections import deque
from twilio.rest import Client

# Ensure 'outputs' directory exists
if not os.path.exists("outputs"):
    os.makedirs("outputs")

# Check required files
for file in ["yolov4.weights", "yolov4.cfg", "coco.names"]:
    if not os.path.exists(file):
        print(f"Error: {file} not found!")
        exit()

# Load YOLO model
weights_path = "yolov4.weights"
config_path = "yolov4.cfg"
labels_path = "coco.names"

with open(labels_path, "r") as f:
    labels = f.read().strip().split("\n")

net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
layer_names = net.getLayerNames()
out_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers().flatten()]

# Detect persons using YOLO
def detect_person_yolo(frame, conf_threshold=0.5, nms_threshold=0.4):
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(out_layers)

    boxes, confidences = [], []
    person_detected = False
    
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if labels[class_id] == "person" and confidence > conf_threshold:
                box = detection[0:4] * np.array([width, height, width, height])
                (centerX, centerY, w, h) = box.astype("int")
                x = int(centerX - w / 2)
                y = int(centerY - h / 2)
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                person_detected = True
    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    
    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f'Person {confidences[i]:.2f}', (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    print(f"Person detected: {person_detected}")
    return person_detected, frame

# Twilio SMS function
def load_credentials():
    try:
        with open("credentialss.txt", "r") as f:
            return json.load(f)
    except Exception as e:
        print(f"Error loading credentials: {e}")
        return None

# Load credentials once at the start
info_dict = load_credentials()
if not info_dict:
    print("Error: Could not load Twilio credentials! Exiting...")
    exit()  # Stop execution if credentials are missing

print("Loaded Twilio credentials successfully:", info_dict)  # Debugging

def send_message(body):
    try:
        account_sid = info_dict['account_sid']
        auth_token = info_dict['auth_token']
        client = Client(account_sid, auth_token)
        print(f"Using Twilio SID: {account_sid}")
        print(f"Using Twilio From Number: {info_dict['trial_num']}")
        print(f"Sending SMS to: {info_dict['your_num']}")
        message = client.messages.create(